In [132]:
### IMPORT AND CONFIGURATION
import numpy as np
import torch
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import matplotlib.pyplot as plt
from scipy.optimize import fsolve


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
vocab = tokenizer.get_vocab()
model = GPT2Model.from_pretrained("gpt2", output_hidden_states=True, output_attentions=True)
embeddings = model.wte.weight.data

def get_tokens_prob(x, k=5):
    # Compute the similarity between the token and all the tokens in the vocabulary, then applies softmax
    prob = torch.softmax(torch.matmul(x,embeddings.T), dim=-1)
    # Get the top k tokens
    top = torch.topk(prob, k=k)
    idxs = top.indices
    tokens = [tokenizer.decode(idx) for idx in idxs]
    return top.values, idxs, tokens

## Applying MaxEnt to explore the inner working of GPT2 

Let $\vec e_1, \vec e_2, \dots, \vec e_{n+1}$ be the tokens in the input prompt. We'd like to determine:
$$
p(\vec x | \vec e_1, \vec e_2, \dots, \vec e_n)
$$
which is essentially what the transformer computes after $12$ repetitions of the ATTN+FFNN mechanism. In principle, since the action of the transformer (_the dynamical rule_) is known, one could potentially figure out exactly the resulting pdf in the vocabulary space. This is however analytically infeasible, since the transformer is made of several non linear layers (FFNN) and the dynamics is allegedely extremely complicated. Thus, analoguosly to what we do in stat mech, we hope our system is ergodic and try to build a statistical description on top of the whole embedding space.

 In particular, we resort to the __MaxEnt principle__ to compute (analytically or not) the final conditioned pdf. Note that we do have the "correct" pdf $p(\vec x | \vec e_1, \vec e_2, \dots, \vec e_n)$. We just want to see whether, upon applying MaxEnt with the right constraints, we can obtain something similar to the distribution computed by the transformer

 ### Bolztmann-like approach
 The entropy functional on reads:

$$
S[p] = -k \int d^D x \> p(\vec x) \ln p(\vec x) 
$$
the first constraint is, obviously, the normalization $ \int d^D x \> p(\vec x) = 1 $

In this first paragraph, we apply the following constraint:
$$
\langle \vec x \rangle = \vec{e}_{n+1}
$$
because we do have the "right" next token (again, we don't want to perform prediction here with the maxEnt. We just wish to see if the resulting pdf has some similarities with the one suggested by the maximization of $S$).

This leads to an analytic form for $p(\vec x | \vec e_1, \vec e_2, \dots, \vec e_n)$:
$$
p(\vec x | \vec e_1, \vec e_2, \dots, \vec e_n) = p(\vec x) = \frac{1}{Z} \exp(\vec \mu \cdot \vec x)
$$
with $Z =\prod_{i=1}^{D} \frac{2 sinh(\mu_i)}{\mu_i}$ and the vector $\mu$ (the lagrange multiplier) is such that:
$$
\vec e_{n+1} = -\frac{1}{\vec\mu} + \frac{1}{\tanh(\vec\mu)} \Longleftrightarrow \vec\mu = f(\vec e_{n+1})
$$

Practically speaking, we have built a pdf Boltzmann-like where the expected value has to be the correct next token and the mode (the vector in the embedding space that maximizes the probability) is precisely $\vec \mu$. Let's see what $\vec \mu$ is

In [130]:
# This is a multipurpose function that can be used to extract the probability density function of the next token and will be useful later.
def GPT2extractPDFLastToken(prompt, lengthPrediction=5, returnTensors = False):
    """
    This function takes a prompt masking the last word and returns the probability density function as computed by GPT 2 of what was the masked token.
    Additionally, it returns the attention vector and the embedding matrix of the masked sequence of tokens, if needed.
    :param prompt: The input prompt
    :return: The probability density function of the next token
    """
    prompt = ' '.join(prompt.split()[:-1])
    inputs = tokenizer(prompt, return_tensors="pt")
    n = len(inputs["input_ids"][0]) 

    with torch.no_grad():
        # run the transfomer inference
        outputs = model(**inputs)
        hidden_states = outputs.hidden_states
        # get the hidden states (i.e. matrices of the last layer)
    layers = hidden_states[1:]
    # extract the attentions matrices
    attentions = outputs.attentions

    # the next token is the last one in the last layer
    nextToken = (layers[-1][0])[-1]
    # project it into the vocabulary space and build a pdf on top of it using softmax
    prob, idxs, tokens = get_tokens_prob(nextToken, lengthPrediction)

    if (returnTensors):
        chosenLayer = 0
        lastStepAttentionMatrices = attentions[chosenLayer][0]
        vectorA = torch.tensor( np.zeros(n-1) )              # Not the last token (itself w/ itself)
        #iterate on the heads
        for head in range(12):
            attentionHead = lastStepAttentionMatrices[head]
            column = attentionHead[-1,:]   # last row
            column = column[:-1]           # remove the last element (autocorrelation)
            vectorA = (vectorA + column)
        # Normalization step
        vectorA = vectorA / np.linalg.norm(vectorA) 
        T = torch.nn.functional.one_hot(inputs.input_ids[0], num_classes=embeddings.shape[0]).float()
        E = torch.matmul(T,embeddings)
        E = E / np.linalg.norm(E, axis=1, keepdims=True)     # Normalize again by row
        E = E[:-1,:]                                         # remove the last token
        return prob, idxs, tokens, vectorA, E
    else:
        return prob, idxs, tokens

In [131]:
prompt = "I'm an Italian physicist who lived in Padua. I am known for my work in the field of optics and astronomy. " \
        "I was born in 1564 and died in 1642. My name is Galileo Galilei"
prob, idxs, tokens = GPT2extractPDFLastToken(prompt)
for p, idx, token in zip(prob, idxs, tokens):
    print(f" -> logp = {-torch.log(p):.2f}: ['{token}'], idx = {idx}")

 -> logp = 0.28: [' Galile'], idx = 32422
 -> logp = 2.29: ['.'], idx = 13
 -> logp = 3.12: [' and'], idx = 290
 -> logp = 3.55: [','], idx = 11
 -> logp = 5.17: ['."'], idx = 526


In [141]:
#This is the function we want to solve
def f(x,k):
    return 1 / np.tanh(x) - 1 / x - k

# We need to constraint the mean value of the distribution to be equal to the expected value of the token
tokenExpected = embeddings[idxs[0]]      # extracted from above

# Solve it component by component
mu = torch.tensor(np.zeros(embeddings.shape[1]), dtype=torch.float32)
for i,d in enumerate(tokenExpected):
    xi = d.item()
    mu_i = fsolve(f, 1, args=(xi))
    mu[i] = (mu_i.item())

print(f"Succesfully computed mu vector.")
print("It represents the direction of maximum probability; projecting mu over the vocabolary we get: \n")
# mu should be the direction of maximum probability (mode). At what does it point to?
prob, idxs, tokens = get_tokens_prob(mu, 8)
for p, idx, token in zip(prob, idxs, tokens):
    print(f" -> logp = {-torch.log(p):.2f}: ['{token}'], idx = {idx}")

print("\nAs expected, it points practically with certainty to the right expected token")

Succesfully computed mu vector.
It represents the direction of maximum probability; projecting mu over the vocabolary we get: 

 -> logp = -0.00: [' Galile'], idx = 32422
 -> logp = 38.92: [' Canaver'], idx = 46858
 -> logp = 40.84: ['Palest'], idx = 32570
 -> logp = 40.95: [' Galileo'], idx = 45860
 -> logp = 41.26: [' Presbyter'], idx = 40507
 -> logp = 41.34: [' Palest'], idx = 5480
 -> logp = 41.82: [' Jude'], idx = 30044
 -> logp = 42.00: [' horizont'], idx = 13736

As expected, it points practically with certainty to the right expected token


This is a boring result. We did get the right token but this was, of course, by no surprise since we cooked that information into the pdf. In fact, the other tokens suggested by the Boltzmann model were probably related to the word "Galilea" (Palest, Jude, Canaver). Since we didn't used the previous token, we lost all the context contained by the whole sequence of the prompt





DA AGGIUNGERE: LOG-LIKELI ESPONENZIALE COL PRODOTTO SCALARE, DA AGGIUNGERE? GIUSTIFICAZIONE A POSTERIORI


### Using the attention matrix

The previous approach was clearly extremely simple and trivial, lacking precious information that we have on the resulting pdf (in fact, we didn't use the previous tokens $\{e_i\}$). Let us think of a new constraint. Since the dot product in the embedding space (upon normalization) is somehow related to the similarities between tokens, we may wish to constrain:
$$ 
\langle \vec x \cdot \vec e_i \rangle = something, \>\> \forall \vec e_i
$$
Since the transfomer provides us with attention matrices, why don't use those quantities to constrain our entropy. In particular, if $\hat A$ is the $n+1 \times n+1$ attention matrix at the end of 

In [142]:
# f function (to improve stability, Taylor expansion around 0)
def f(z, tol = 0.1):
    z = np.asarray(z) 
    result = np.empty_like(z, dtype=float)
    # when z is small, we use a Taylor expansion (to avoid numerical instability)
    z_small = z[np.abs(z) < tol]
    result[np.abs(z) < tol] = (-1/3)*z_small + (2/45)*z_small**3 - (17/945)*z_small**5
    # when z is big, we use the original function
    z_big = z[np.abs(z) > tol]
    result[np.abs(z) > tol] = - 1 / np.tanh(z_big) + 1 / z_big
    return result

# This is the system we want to solve
def system(x, E, A):
    z = np.dot(E.T, x)     
    fz = f(z)        
    fz = fz.reshape(-1, 1)     
    v = E @ fz
    v = v.reshape(-1)      
    return v - A      

def solveMu(E, vectorA, amplificationAttention=1):
    x0 = np.random.random(len(vectorA))      # incredibly sensible to the initial condition
    x0 = x0 / np.linalg.norm(x0)             # normalizing the initial condition
    mu = fsolve(system, x0, args=(E.numpy(), amplificationAttention*vectorA.numpy()))
    return mu

In [143]:
prompt = "The american flag is white, red and blue. The flag has 13 stripes and 50 stars"
prompt = "Mount Everest is the tallest mountain on Earth, located in the Himalayas on the border between Nepal and China"
prompt = "DNA, or deoxyribonucleic acid, is the molecule that carries genetic instructions in all living organisms. It is composed of four nucleotide bases: adenine, thymine, cytosine, and guanine. These bases form sequences that determine traits and biological functions."
#prompt = "The year is 2194, and humanity no longer lives on Earth. After the Collapse, we fled to floating arcologies orbiting the gas giants. Every child knows the stories of our homeworld, but none of us have seen a tree, felt rain, or walked on soil. Until today."
prompt = "I'm an Italian physicist who lived in Padua. I am known for my work in the field of optics and astronomy. " \
        "I was born in 1564 and died in 1642. My name is Galileo"
prompt = "The american flag is white, red and blue"
prompt = "I'm a German-born theoretical physicist who revolutionized modern physics with my theory of relativity. " \
"I was awarded the Nobel Prize in Physics in 1921 for my explanation of the photoelectric effect." \
" I was born in 1879 and died in 1955. My name is Albert Einstein"

prob, idxs, tokens = GPT2extractPDFLastToken(prompt)
print("Masking the last token (\"" , ''.join(prompt.split()[-1]), "\"), GPT2 would have predicted:", sep = "")
for p, idx, token in zip(prob, idxs, tokens):
    print(f" -> -logP = {-torch.log(p):.2f}: ['{token}'], idx = {idx}")

# Extract the needed matrices from the GPT2 model (embedding matrix and attention vector)
_,_,_, vectorA, E = GPT2extractPDFLastToken(prompt, returnTensors=True)
# Let's solve for mu, now that we have the attention vector and the embedding matrix
amplificationAttention = 1
mu = solveMu(E, vectorA, amplificationAttention)
# given mu, we can compute the mean and the mode according to the maxent model
x_avg, x_mode = f(E.T @ mu), -E.T @ mu

print("\nA MaxEnt approach yields:")
print("For the mean value (MAXENT):")
prob, idxs, tokens = get_tokens_prob(torch.tensor(x_avg, dtype = torch.float), 10)
for p, _, token in zip(prob, idxs, tokens):
    print(f" -> {-torch.log(p):.2f}: '{token}'")

print("For the mode value (MAXENT):")
prob, idxs, tokens = get_tokens_prob(torch.tensor(x_mode, dtype = torch.float), 8)
for p, _, token in zip(prob, idxs, tokens):
    print(f" -> {-torch.log(p):.2f}: '{token}'")

expectedA = E @ x_avg
print( "\n\nError on A: ", (expectedA/amplificationAttention - vectorA)/vectorA, "\n\n")

Masking the last token ("Einstein"), GPT2 would have predicted:
 -> -logP = 0.58: [' Einstein'], idx = 24572
 -> -logP = 4.39: [' B'], idx = 347
 -> -logP = 4.67: [' von'], idx = 18042
 -> -logP = 4.79: [' Hof'], idx = 37745
 -> -logP = 4.83: ['.'], idx = 13

A MaxEnt approach yields:
For the mean value (MAXENT):
 -> 10.19: ' physicist'
 -> 10.29: ' mathemat'
 -> 10.32: ' physicists'
 -> 10.34: ' Nobel'
 -> 10.36: ' relativity'
 -> 10.40: ' chemist'
 -> 10.40: ' mathematician'
 -> 10.41: ' 1955'
 -> 10.42: ' Einstein'
 -> 10.42: ' Physics'
For the mode value (MAXENT):
 -> 8.93: ' physicist'
 -> 9.24: ' mathemat'
 -> 9.34: ' physicists'
 -> 9.39: ' Nobel'
 -> 9.45: ' relativity'
 -> 9.56: ' chemist'
 -> 9.57: ' mathematician'
 -> 9.60: ' 1955'


Error on A:  tensor([-0.3714,  0.7095,  0.1139, -0.0253,  0.7658,  0.1468,  0.1167, -0.1898,
         0.1799,  0.0827,  0.0938, -0.1063,  0.4455,  0.7829,  0.1868, -0.1405,
         0.3817, -0.0531,  1.1250,  1.2763,  1.6499, -0.1436,  0.0263,  

/tmp/ipykernel_6486/785929897.py:25: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  mu = fsolve(system, x0, args=(E.numpy(), amplificationAttention*vectorA.numpy()))
/tmp/ipykernel_6486/3445847984.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prob, idxs, tokens = get_tokens_prob(torch.tensor(x_mode, dtype = torch.float), 8)


The expected value is not that bad (comments)

Nota bene una cosa: qua stiamo ponendo solo $n$ condizioni, ma la distribuzione è su di uno spazio $ d > n $ dimensionale. Dunque stiamo dicendo che il valore medio \langle x \rangle non è completamente vincolato (come nel caso precedente), ne stiamo vincolando solo una parte ($n$ componenti, linearmente). Il resto fa maxEnt. Il fatto che qua il valore medio venga decente, almento secondo me, non è scontato (soprattutto se $n$ è molto più piccolo di $d$).

Sarebbe carino capire come migliora il modello aggiungendo vincoli, cioè aumentando la dimensione del prompt.

----
Let's explore a bit the linear approximation

In [144]:
# Gram matrix
G = E @ E.T
print("The det(G) is: ", np.linalg.det(G))

if (np.linalg.matrix_rank(G) == G.shape[0]):
    mu_linear = np.linalg.solve(G, -3.*vectorA)
    x_avg, x_mode = E.T @ vectorA, np.linalg.inv(G) @ mu_linear

    print("Given prompt: \"", ' '.join(prompt.split()[:-1]), "\"")
    print("Correct next token: ", ''.join(prompt.split()[-1]))
    print("\nExpected value (MAXENT):")
    prob, idxs, tokens = get_tokens_prob(torch.tensor(x_avg, dtype = torch.float), 10)
    for p, _, token in zip(prob, idxs, tokens):
        print(f" -> {-torch.log(p):.2f}: '{token}'")

    print("Mode value (MAXENT):")
    prob, idxs, tokens = get_tokens_prob(torch.tensor(x_mode, dtype = torch.float), 8)
    for p, _, token in zip(prob, idxs, tokens):
        print(f" -> {-torch.log(p):.2f}: '{token}'")

    expectedA = E @ x_avg
    #print( "\n\nError on A: ", (expectedA/amplificationAttention - vectorA)/vectorA, "\n\n")
else:
    x_avg = E.T @ vectorA.float()
    print("Given prompt: \"", ' '.join(prompt.split()[:-1]), "\"")
    print("Correct next token: ", ''.join(prompt.split()[-1]))
    print("\nExpected value (MAXENT):")
    prob, idxs, tokens = get_tokens_prob(torch.tensor(x_avg, dtype = torch.float), 10)
    for p, _, token in zip(prob, idxs, tokens):
        print(f" -> {-torch.log(p):.2f}: '{token}'")


The det(G) is:  -0.0
Given prompt: " I'm a German-born theoretical physicist who revolutionized modern physics with my theory of relativity. I was awarded the Nobel Prize in Physics in 1921 for my explanation of the photoelectric effect. I was born in 1879 and died in 1955. My name is Albert "
Correct next token:  Einstein

Expected value (MAXENT):
 -> 5.77: ' mathemat'
 -> 7.50: ' challeng'
 -> 7.54: ' rul'
 -> 7.65: ' nodd'
 -> 7.65: ' horizont'
 -> 7.68: ' arrang'
 -> 7.68: ' advoc'
 -> 7.71: ' distingu'
 -> 7.71: ' destro'
 -> 7.73: ' defic'


/tmp/ipykernel_6486/2014146491.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prob, idxs, tokens = get_tokens_prob(torch.tensor(x_avg, dtype = torch.float), 10)
